In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('../../BadNets/datasets/usts/Images/0003658.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
''' Multi-GPU Training Example.

Train a convolutional neural network on multiple GPU with TensorFlow.

This example is using TensorFlow layers, see 'convolutional_network_raw' example
for a raw TensorFlow implementation with variables.

This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import division, print_function, absolute_import

import numpy as np
import tensorflow as tf
import time

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Training Parameters
num_gpus = 2
num_steps = 200
learning_rate = 0.001
batch_size = 1024
display_step = 10

# Network Parameters
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units


# Build a convolutional neural network
def conv_net(x, n_classes, dropout, reuse, is_training):
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, 28, 28, 1])

        # Convolution Layer with 64 filters and a kernel size of 5
        x = tf.layers.conv2d(x, 64, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        x = tf.layers.max_pooling2d(x, 2, 2)

        # Convolution Layer with 256 filters and a kernel size of 5
        x = tf.layers.conv2d(x, 256, 3, activation=tf.nn.relu)
        # Convolution Layer with 512 filters and a kernel size of 5
        x = tf.layers.conv2d(x, 512, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        x = tf.layers.max_pooling2d(x, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
        x = tf.contrib.layers.flatten(x)

        # Fully connected layer (in contrib folder for now)
        x = tf.layers.dense(x, 2048)
        # Apply Dropout (if is_training is False, dropout is not applied)
        x = tf.layers.dropout(x, rate=dropout, training=is_training)

        # Fully connected layer (in contrib folder for now)
        x = tf.layers.dense(x, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        x = tf.layers.dropout(x, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(x, n_classes)
        # Because 'softmax_cross_entropy_with_logits' loss already apply
        # softmax, we only apply softmax to testing network
        out = tf.nn.softmax(out) if not is_training else out

    return out


def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads


# By default, all variables will be placed on '/gpu:0'
# So we need a custom device function, to assign all variables to '/cpu:0'
# Note: If GPUs are peered, '/gpu:0' can be a faster option
PS_OPS = ['Variable', 'VariableV2', 'AutoReloadVariable']

def assign_to_device(device, ps_device='/cpu:0'):
    def _assign(op):
        node_def = op if isinstance(op, tf.NodeDef) else op.node_def
        if node_def.op in PS_OPS:
            return "/" + ps_device
        else:
            return device

    return _assign


# Place all ops on CPU by default
with tf.device('/cpu:0'):
    tower_grads = []
    reuse_vars = False

    # tf Graph input
    X = tf.placeholder(tf.float32, [None, num_input])
    Y = tf.placeholder(tf.float32, [None, num_classes])

    # Loop over all GPUs and construct their own computation graph
    for i in range(num_gpus):
        with tf.device(assign_to_device('/gpu:{}'.format(i), ps_device='/cpu:0')):

            # Split data between GPUs
            _x = X[i * batch_size: (i+1) * batch_size]
            _y = Y[i * batch_size: (i+1) * batch_size]

            # Because Dropout have different behavior at training and prediction time, we
            # need to create 2 distinct computation graphs that share the same weights.

            # Create a graph for training
            logits_train = conv_net(_x, num_classes, dropout,
                                    reuse=reuse_vars, is_training=True)
            # Create another graph for testing that reuse the same weights
            logits_test = conv_net(_x, num_classes, dropout,
                                   reuse=True, is_training=False)

            # Define loss and optimizer (with train logits, for dropout to take effect)
            loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=logits_train, labels=_y))
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            grads = optimizer.compute_gradients(loss_op)

            # Only first GPU compute accuracy
            if i == 0:
                # Evaluate model (with test logits, for dropout to be disabled)
                correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.argmax(_y, 1))
                accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

            reuse_vars = True
            tower_grads.append(grads)

    tower_grads = average_gradients(tower_grads)
    train_op = optimizer.apply_gradients(tower_grads)

    # Initialize the variables (i.e. assign their default value)
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    #sess = tf.Session(config=config)
    
    init = tf.global_variables_initializer()
    

    # Start Training
    with tf.Session(config=config) as sess:

        # Run the initializer
        sess.run(init)

        # Keep training until reach max iterations
        for step in range(1, num_steps + 1):
            # Get a batch for each GPU
            batch_x, batch_y = mnist.train.next_batch(batch_size * num_gpus)
            # Run optimization op (backprop)
            ts = time.time()
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
            te = time.time() - ts
            if step % display_step == 0 or step == 1:
                # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                     Y: batch_y})
                print("Step " + str(step) + ": Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc) + ", %i Examples/sec" % int(len(batch_x)/te))
            step += 1
        print("Optimization Finished!")

        # Calculate accuracy for MNIST test images
        print("Testing Accuracy:", \
            np.mean([sess.run(accuracy, feed_dict={X: mnist.test.images[i:i+batch_size],
            Y: mnist.test.labels[i:i+batch_size]}) for i in range(0, len(mnist.test.images), batch_size)]))
        sess.close()

In [ ]:
import pickle
import numpy as np
import pprint

train_file = '../../BadNets/datasets/usts/pickles/clean_set_trn.pkl'
test_file = '../../BadNets/datasets/usts/pickles/clean_set_tst.pkl'
dict_file = '../../BadNets/datasets/usts/pickles/images_dict.pkl'

file_list = {'training':train_file,'testing':test_file,'images':dict_file}

all_items = {}

for k,v in file_list.iteritems():
    with open(v,'rb') as f:
        all_items[k] = pickle.load(f);
        


In [ ]:
pp = pprint.PrettyPrinter(indent=4,depth=4)
pp.pprint(all_items['testing'])

In [ ]:
from __future__ import division
from scipy.stats import hypergeom
from scipy.special import comb
from math import ceil,isnan
import matplotlib.pyplot as plt
#import pixiedust
import sys
from decimal import *
import logging

BATCH_SIZE = 128 # 20
POISONED_SIZE = 10 # 5
CLEAN_SIZE = 50000 # 200

logger = logging.getLogger(__name__)
log_file_name = 'B_'+str(BATCH_SIZE)+'_POIS_'+str(POISONED_SIZE)+'_CLEAN_' + str(CLEAN_SIZE)+'.log'
logging.basicConfig(filename=log_file_name,level=logging.DEBUG,append=False)

def calc_gt_a(C,d,k,a=1.0):
    dist = hypergeom(C,d,k)
    return 1.0 - dist.cdf(a)

def prb_d_compose(N,d,C):
    logger.info("[[ prb_d_compose called with (N,d,C) = (%d,%d,%d) ]]" , N,d,C)
    #print ("prb_d_compose is called with N=%d,d=%d,C=%d" % (N,d,C))
    bottom = comb(N+C-1,C-1,exact=True)
    #print ("bottom is: %d" % (bottom))
    if bottom == 0.0 :
        return 0.0
    result = Decimal(comb(N-1,d-1,exact=True))*Decimal(comb(C,d,exact=True))/Decimal(bottom)
    #print (result)
    #if isnan(result):
    #        print ("Is nan is true N,d,C ",  (N,d,C))
    #        sys.exit()
    return result
    
def calc_expect(N,C,k):
    logger.info("[ calc_expect called with (N,C,k) = (%d,%d,%d) ]" , N,C,k)
    summ = Decimal(0.0)
    ten_percent = int(0.1 * N)
    percent = 10
    for i in range(1,N+1):
        if i%ten_percent == 0:
            #print ("%d percent..."%(percent))
            #logger.info("[[ %d percent ]]" , percent)
            percent = percent + 10
        prb_cmps = prb_d_compose(N,i,C)
        if Decimal(prb_cmps) == Decimal('0.0') :
            continue
        summ = summ+(Decimal(calc_gt_a(C,i,k))*prb_cmps)
    logger.info("[ calc_expect called with (N,C,k) = (%d,%d,%d) finished with %f]" , N,C,k,summ)
        #if isnan(summ):
        #    print ("Is nan is true N,C,k",  (N,C,k))
        #    sys.exit()
    return summ

def transform_eq(epochs,batch_size,poisoned_size,clean_size):
    total_data_size = clean_size + poisoned_size
    epoch_num_batches = int(ceil(total_data_size / batch_size)) + 1
    C = epochs * epoch_num_batches
    P = epochs * poisoned_size
    N = (batch_size * C) - P
    
    return N,C
        
        

In [ ]:
#%%pixie_debugger
logger.info("+++")
logger.info("batch size, poisoned_size and clean size are (%d, %d, %d)",BATCH_SIZE,POISONED_SIZE,CLEAN_SIZE)
EPOCHS = [1,5,10,20,30,50,100]
PROP = [0.01, 0.02, 0.05, 0.1,0.25]
#PROP = [0.01]
epoch_res = []
for epc in EPOCHS:
    N,C = transform_eq(epc,BATCH_SIZE,POISONED_SIZE,CLEAN_SIZE)
    logger.info("***")
    logger.info("*N and C are %d, %d for epoch %d",N,C,epc)
    Ks = [int(C*k) for k in PROP] 
    Results = [float(calc_expect(N,C,k)) for k in Ks]
    print Results
    logger.info(Results)
    epoch_res.append(Results)
    logger.info( "*done for epoch %d" , (epc))

In [ ]:
from __future__ import division
xxx = comb(49944+398-1,398-1,exact=True)/306388518658647242127450100000068342277049800000008069685154164976613796257102259101090365047307606957393363425505327668702850647686242466682782398444469981491861944363527509601813215292993574698538719125310445733834431046937583132110550304253782713323719637615853863065023785092162322145059589507812468257364153527234231029035670730169595950611218898405201808382370578885498984646440003878439247928258077223258502933643893692162034347213283138155580438258223514796469686242431363246253276629312529808343722196481017886291511007764670506344307062435264234707495609506743647157432456599306877780873148033086652088281163249163082382429421147098171065476349613207400998848302365210558096233600289363423172937933132035165383555314173021324225572229518650812604397117999838049743572411848261763229037064733722665879577320430888959202492143486426423078109446814918778521100286194365356170171497737832900350000000000000000001431106553500000000000059788566988016340011002400724318162476853886425245713213540190089023238166900L
print (xxx)

In [ ]:
Decimal(399) / Decimal(332859656806308444463895176214129937185404287342189283714769802893282973376704827856456811337914388690004972911760860138382837343841345376907252170925210524281256154677257805136231369551365038160344180441046891754482335647223614017034426601794053540077744282898410873983413476773057718429179545300579406296293688549102038714366333705985931156622025165352195960563961683958095987444658111674682097233605367094421093259289069247976338201876884268972765597411779807152488864299592990681167129537855457476525567853903146578534876651803038028193902369692053048050533980143304284764676540737344335026069033589977519311585734924218101767639962090164747347932509329137052032994737514913470001518387895854777578576992305079708960595093884972534279159896644194790632384825180615950827089053103652004164140788880079287347806357325860000581520301940247102977259499383879585162378990493132589814402065197105292567102811152255751346023693524350808414883382302356940987856824454794547831939167873423859892457416420966362400)

In [ ]:
print Result[0]